In [95]:
import pandas as pd
from scipy import stats
import numpy as np

In [69]:
headlines = pd.read_csv('combined_headlines.csv')
sentiment = pd.read_csv('headline_sentiment.csv')

In [70]:
#Last day with sentiment data - 899516
#sentiment values 903935
#headlines values 905001
sentiment

,Unnamed: 0,date,titles,wordCount,sentimentGI,negativityGI,positivityGI,sentimentHE,negativityHE,positivityHE,sentimentLM,negativityLM,positivityLM,ratioUncertaintyLM,sentimentQDAP,negativityQDAP,positivityQDAP,direction
0,0,2006-01-01,Russia completes Ukraine gas cut-off,5,0.200000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,neutral
1,1,2006-01-01,High winds cause havoc,4,-0.250000,0.250000,0.000000,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,-0.250000,0.250000,0.000000,negative
2,2,2006-01-01,Quake sparks panic in Aceh,4,-0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,-0.250000,0.250000,0.000000,0.000000,-0.250000,0.250000,0.000000,negative
3,3,2006-01-01,Russia takes over G8,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,neutral
4,4,2006-01-01,Turkish children tested for bird flu,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,neutral
5,5,2006-01-01,Baghdad bomb attacks injure 17,4,-0.750000,0.750000,0.000000,0.000000,0.000000,0.000000,-0.250000,0.250000,0.000000,0.000000,-0.750000,0.750000,0.000000,negative
6,6,2006-01-01,Foreigner kidnapped in Gaza Strip,4,-0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,neutral
7,7,2006-01-01,Adelaide notch 4-2 win against Roar,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000,0.000000,0.250000,0.000000,0.250000,positive
8,8,2006-01-01,"Famine a national disaster, Kenyan President says",6,-0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,-0.166667,0.166667,0.000000,0.000000,-0.333333,0.333333,0.000000,negative
9,9,2006-01-01,Adelaide trounce Roar to extend lead,5,0.200000,0.000000,0.200000,0.200000,0.000000,0.200000,0.200000,0.000000,0.200000,0.000000,0.200000,0.000000,0.200000,positive


# MODE

In [77]:
sentiment['date'] = pd.to_datetime(sentiment['date'])
data = sentiment.copy()
data.set_index(data['date'], inplace=True)
data = data.dropna()

In [78]:
days = data.groupby([(data.index.date),data['direction']]).size().reset_index()

In [79]:
counts = pd.DataFrame(days.loc[days.groupby([days['level_0']])[0].idxmax()]).reset_index(drop=True)

In [107]:
modes = counts.drop([0], axis=1)
modes = modes.rename(index=str, columns={'level_0': 'date', 'direction': 'mode'})

In [111]:
modes.to_csv('sentiment_modes.csv', index=False)

# AVERAGE SUM

In [122]:
days2 = data.groupby([(data.index.date)]).sum().reset_index()
days2 = days2.rename(index=str, columns={'index': 'date','sentimentQDAP':'Sum_QDAP'})
sums = days2[['date','sentimentGI','sentimentHE','sentimentLM','Sum_QDAP']]
sums = sums.copy()
sums['Sum'] = np.where(sums['Sum_QDAP'] < 0, 'negative', 
                                (np.where(sums['Sum_QDAP'] > 0, 'positive', 'neutral')))

In [123]:
days3 = data.groupby([(data.index.date)]).mean().reset_index()
days3 = days3.rename(index=str, columns={'index': 'date', 'sentimentQDAP': 'Mean_QDAP'})
means = days3[['date','sentimentGI','sentimentHE','sentimentLM','Mean_QDAP']]
means = means.copy()
means['Mean'] = np.where(means['Mean_QDAP'] < 0, 'negative', 
                                (np.where(means['Mean_QDAP'] > 0, 'positive', 'neutral')))

In [112]:
sums.to_csv('sentiment_sums.csv', index=False)

In [113]:
means.to_csv('sentiment_means.csv', index=False)

# Combine

In [131]:
sums2 = sums[['Sum_QDAP','Sum']]
means2 = means[['Mean_QDAP','Mean']]
combined = pd.concat([modes,sums2, means2], axis=1)

In [134]:
combined.to_csv('sentiment_days.csv', index=False)